# Student Performance - Feature Engineering

## Set Up Environment

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
scores = pd.read_csv('StudentsPerformance.csv')
scores.head(3)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93


In [3]:
scores.groupby(['gender', 'race/ethnicity']).mean()

math score  reading score  writing score
gender race/ethnicity                                          
female group A          58.527778      69.000000      67.861111
       group B          61.403846      71.076923      70.048077
       group C          62.033333      71.944444      71.777778
       group D          65.248062      74.046512      75.023256
       group E          70.811594      75.840580      75.536232
male   group A          63.735849      61.735849      59.150943
       group B          65.930233      62.848837      60.220930
       group C          67.611511      65.424460      62.712230
       group D          69.413534      66.135338      65.413534
       group E          76.746479      70.295775      67.394366

## Define Classes

In [4]:
class Group_Metrics:
    def __init__(self, df):
        self.data = df
    
    def insert_group_metric(self, group, metric_col, metric_name, func):
        group_values = {}
        for g in self.data[group].unique():
            subset_df = self.slice_df({group: g})
            value = func(subset_df, metric_col)
            group_values[g] = value
        self.data['{} {} {}'.format(metric_col, group, metric_name)] = self.data[group].map(group_values)
    
    def _slice_df(self, filters):
        cols = list(filters)
        vals = list(filters.values())
        subset_df = self.data
        for i in range(len(filters)):
            subset_df = subset_df[subset_df[cols[i]] == vals[i]]
        return subset_df
    
    def _calc_mean(self, df, col):
        return round(df[col].mean(), 1)
    
    def _calc_median(self, df, col):
        return df[col].median()
    
    def _calc_min(self, df, col):
        return df[col].min()
    
    def _calc_max(self, df, col):
        return df[col].max()
    
#     def insert_group_metric(self, groups, metric_col, metric_name):
#         group_values = self.data.groupby(groups).mean()[metric_col]
#         group_values = {}
#         for g in self.data[group].unique():
#             subset_df = self.slice_df({group: g})
#             value = func(subset_df, metric_col)
#             group_values[g] = value
#         self.data['{} {} {}'.format(metric_col, group, metric_name)] = self.data[group].map(group_values)
    
    

In [5]:
group_scores = Group_Metrics(scores)

In [6]:
group_scores.data

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [7]:
groups = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
metric_cols = ['math score', 'reading score', 'writing score']
metric_names = ['mean', 'median', 'min', 'max']
functions = [group_scores._calc_mean, group_scores._calc_median, group_scores._calc_min, group_scores._calc_max]

for group in groups:
    for m_col in metric_cols:
        for i in range(4):
            group_scores.insert_group_metric(group, m_col, metric_names[i], functions[i])

AttributeError: 'Group_Metrics' object has no attribute 'slice_df'

In [ ]:
# group_scores.insert_group_metric('gender', 'math score', 'mean', group_scores._calc_mean)

In [ ]:
group_scores.data